In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
lyrics = pd.read_csv("/kaggle/input/bad-bunny-lyrics/bad_bunny_lyrics.csv")

In [3]:
# Good regex resource: https://realpython.com/regex-python/#python-regex-metacharacters
import re

In [4]:
# practice_str = '[Letra de "NI BIEN NI MAL"]  [Intro: Bad Bunny] Yeh-yeh Yeh-yeh Yeh-yeh'
practice_song_lyric = lyrics.iloc[0,4]
practice_song_lyric

In [8]:
print(practice_song_lyric)

In [ ]:
# Source: https://stackoverflow.com/questions/65022050/cleaning-song-lyrics-with-regex

In [39]:
# removed brackets used to denote sections of song
# removed new lines and added periods for sentence tokenization
removed_brackets = re.sub(r"[\[].*?[\]]","",practice_song_lyric)
no_brackets_or_new_lines = re.sub(r"\n{2}","",removed_brackets)
no_brackets_or_new_lines = re.sub(r"[\n]",". ",no_brackets_or_new_lines)
print(no_brackets_or_new_lines)

In [40]:
# might need to replace new lines with periods
correct_vas = re.sub(r"va\'","vas",no_brackets_or_new_lines)
correct_para_zero = re.sub(r"Pa\'l","Para el",correct_vas)
correct_para = re.sub(r"Pa\'","Para ",correct_para_zero)
correct_voy = re.sub(r"vo\'","voy ",correct_para)
correct_para_2 = re.sub(r"pa\'","para",correct_voy)
correct_de = re.sub(r"\'e","de",correct_para_2)
correct_puyas = re.sub(r"puya\'","puyas",correct_de)
correct_puyas += '.'
correct_puyas

0. Sentence Tokenization
0. Finding semantically related words using word embeddings (already loadded the dataset)
0. Spanish lemmatizer - https://stackoverflow.com/questions/60534999/how-to-solve-spanish-lemmatization-problems-with-spacy
    * The link above has sources that show how to make your own lemmatizer, FreeLing (provides lemmatization in Spanish and other languages), and spacy-stanza ("spaCy's API with the Stanza's models")
    * https://github.com/pablodms/spacy-spanish-lemmatizer     
0. Word Cloud
1. Topic Modeling - can later let the user generate songs based on a topic
2. Sentiment Analysis - user can create songs with a specific sentiment
    * Spanish Sentiment Analyis [Library](https://pypi.org/project/sentiment-analysis-spanish/) (uses CNN).

In [28]:
!pip install stanza 

In [29]:
# Source: https://stanfordnlp.github.io/stanza/
# Source: https://stanfordnlp.github.io/stanza/tokenize.html#:~:text=Training%2DOnly%20Options-,Description,invoked%20by%20the%20name%20tokenize%20.
# Note: Stanza does not offer a Spanish Sentiment Analyzer
# Check out Stanza's other sections like:
# - Pipeline and Processors
# - Part-of-Speech and Morphological Features
# - Lemmatization
import stanza

In [30]:
stanza.download('es', package='ancora', processors='tokenize,mwt,pos,lemma', verbose=True)

In [50]:
stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma', lang='es', use_gpu=True)

In [32]:
doc = stNLP('Barack Obama nació en Hawaii.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [41]:
# Source: https://stanfordnlp.github.io/stanza/tokenize.html#:~:text=Training%2DOnly%20Options-,Description,invoked%20by%20the%20name%20tokenize%20.
# tokenization and sentence segmentation performed
doc = stNLP(correct_puyas)

In [45]:
for i, sentence in enumerate(doc.sentences[4:6]):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [47]:
print([sentence.text for sentence in doc.sentences[4:6]])

In [44]:
doc.sentences[4:5]

In [ ]:
# replaces all Unicode whitespace (such as \u) with a single space
#re.sub("\s+", " ", lyric, flags=re.UNICODE)

In [ ]:
# Spanish lemmatization (pattern)
# Spanish word normalization Python packages and libraries (Cucco)
# ftfy - handles mojibake
# Tokenize - nltk package (tokenize and FreqDist)